<a href="https://colab.research.google.com/github/SpencerPao/Natural-Language-Processing/blob/main/BERT/BERT_Code_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Clone Repository](https://github.com/google-research/bert) from Google Research
- Pretty much has everything that you need to get started on training and utilizing BERT
- A good chunk of this notebook (BERT) has been utilized from [this notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

In [ ]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 8.42 MiB/s, done.
Resolving deltas: 100% (182/182), done.


In [ ]:
# We want to use 1.X
% tensorflow_version 1.X

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.X`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
'''Load in our models and important packages...'''
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tensorflow as tf
from datetime import datetime

In [ ]:
print(tf.__version__)

1.15.2


In [ ]:
!pip install bert-tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.9 MB/s 


In [ ]:
import sys
sys.path.append("/content/bert")

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# Let's get a base BERT Model

In [ ]:
# we'll use the base model uncased since Text is lowercased. (uncased just lowercases all incoming raw text)
# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Simiarly, we can get the models straight from the source...


# Download BERT BASE model from tF hub 

# Instructions on how to get literal model weights.

# !curl https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip --output uncased_L-12_H-768_A-12.zip
# !unzip uncased_L-12_H-768_A-12.zip

# If you are on windows and on local machine.
# !python -m wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# import zipfile
# with zipfile.ZipFile('uncased_L-12_H-768_A-12.zip', 'r') as zip_ref:
#     zip_ref.extractall('bert/content/') # zipping files to this location.

In [ ]:
# Reading in twitter data on sentiment. (NEGATIVE, POSITIVE for target)
# Already cleaned and preprocessed...
df = pd.read_csv('twitter_data.csv')
df = df.sample(frac=1).reset_index()
df = df.drop(['index'], axis = 1)
df

,target,ids,date,flag,user,text
0,POSITIVE,1557045453,Sun Apr 19 01:42:30 PDT 2009,NO_QUERY,prafulh,hamilton strategy get way top see gravel nice ...
1,NEGATIVE,1551515456,Sat Apr 18 09:15:40 PDT 2009,NO_QUERY,mellen59,good morning 5 30 sleep
2,NEGATIVE,1573640524,Tue Apr 21 02:08:05 PDT 2009,NO_QUERY,natashasaurus,yay one else updating anything
3,POSITIVE,1559874025,Sun Apr 19 12:16:06 PDT 2009,NO_QUERY,JessieValentine,cookie sam tomorrow see wait papa roach concert
4,NEGATIVE,1678266018,Sat May 02 06:07:37 PDT 2009,NO_QUERY,sidSicklePowers,sleepy voice hoarse
...,...,...,...,...,...,...
99995,POSITIVE,1468802525,Tue Apr 07 03:48:52 PDT 2009,NO_QUERY,yoritomo_reiko,okay work afternoon break actually fairly good...
99996,POSITIVE,1469395244,Tue Apr 07 06:14:54 PDT 2009,NO_QUERY,thedudeims,today madame beautiful
99997,NEGATIVE,1556930444,Sun Apr 19 01:04:20 PDT 2009,NO_QUERY,camfitz,dang study break almost already
99998,POSITIVE,1556551070,Sat Apr 18 23:17:32 PDT 2009,NO_QUERY,Kateless,coming tonight


In [ ]:
df.target.unique()

array(['POSITIVE', 'NEGATIVE'], dtype=object)

In [ ]:
decode_map = {"NEGATIVE": 0, "POSITIVE": 1}
def decode_sentiment(label):
    return decode_map[(label)]
df.target = df.target.apply(lambda x: decode_sentiment(x))

In [ ]:
df

,target,ids,date,flag,user,text
0,1,1557045453,Sun Apr 19 01:42:30 PDT 2009,NO_QUERY,prafulh,hamilton strategy get way top see gravel nice ...
1,0,1551515456,Sat Apr 18 09:15:40 PDT 2009,NO_QUERY,mellen59,good morning 5 30 sleep
2,0,1573640524,Tue Apr 21 02:08:05 PDT 2009,NO_QUERY,natashasaurus,yay one else updating anything
3,1,1559874025,Sun Apr 19 12:16:06 PDT 2009,NO_QUERY,JessieValentine,cookie sam tomorrow see wait papa roach concert
4,0,1678266018,Sat May 02 06:07:37 PDT 2009,NO_QUERY,sidSicklePowers,sleepy voice hoarse
...,...,...,...,...,...,...
99995,1,1468802525,Tue Apr 07 03:48:52 PDT 2009,NO_QUERY,yoritomo_reiko,okay work afternoon break actually fairly good...
99996,1,1469395244,Tue Apr 07 06:14:54 PDT 2009,NO_QUERY,thedudeims,today madame beautiful
99997,0,1556930444,Sun Apr 19 01:04:20 PDT 2009,NO_QUERY,camfitz,dang study break almost already
99998,1,1556551070,Sat Apr 18 23:17:32 PDT 2009,NO_QUERY,Kateless,coming tonight


In [ ]:
# If any nulls, replace with empty string.
df["text"].fillna("", inplace = True)
df['text'].isnull().values.any()

False

In [ ]:
# Remember that I did a lot of precleaning to get the dataframe in this state. If you are curious on how I did that,
# the Sentiment Analysis video is here: https://www.youtube.com/watch?v=CzRrD76pnVY&t=785s
df.target.value_counts()

1    50000
0    50000
Name: target, dtype: int64

In [ ]:
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

# We need to convert the Training and Testing Data into a BERT Format
More information can be found [Google AI Example](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb) where I used a good chunk of their code.
- GUID - An id for each row
- text_a is the text we want to classify
- text_b is the sentence we want to predict (used for QA)
- label is the classification of our example (0,1, or true,false)

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# With that in mind, what does our tokenizer do? Let's look at an example
tokenizer.tokenize("Make sure to like and subscribe!") # just breaks up our words into identified vocabulary inside the vocab_file

['make', 'sure', 'to', 'like', 'and', 'sub', '##scribe', '!']

### BERT can't just accept raw text. We have to:
- Add special tokens to separate sentences for classification
- pass sequences of equal length with padding
    - padd with zeroes = pad token and ones = real tokens
- Lowercase text (if using uncased model = lower case)
- Tokenize words
- Break words into pieces if applicable
- Maps words to indices using vocab file (downloaded with the model.zip)
```
# for additional details run this:
!curl https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip --output uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
```



### Important Tokens and Features for BERT
- Special tokens: ([SEP],102) - The marker for the end of a sentence
- Classification: ([CLS],101) - Add this token to the beginning of the sentence so BERT knows we are doing classification
    - Add a linear layer at the end of the model if you have a regression task
- Padding: ([PAD],0)
- Unknown tokens: ([UNK],100)

In [ ]:
# This block helps transform the raw text into a data format that BERT Understands.
DATA_COLUMN = 'text'
LABEL_COLUMN = 'target'
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
label_list = [0,1]
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 80000


INFO:tensorflow:Writing example 0 of 80000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:Writing example 10000 of 20000


INFO:tensorflow:Writing example 10000 of 20000


# Fine Tuning the model!
Steps:
- Loads BERT tf hub module
- Adds a single layer that will be tuned at the end of architecture
- Tune with our purposed data

In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)



In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


# Configuration Block

In [ ]:
OUTPUT_DIR = "bert/output/"
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [ ]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'bert/output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7c37cb8250>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'bert/output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7c37cb8250>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 7500 into bert/output/model.ckpt.


INFO:tensorflow:Saving checkpoints for 7500 into bert/output/model.ckpt.


INFO:tensorflow:Loss for final step: 0.090980425.


INFO:tensorflow:Loss for final step: 0.090980425.


Training took time  1:48:05.261179


# Testing

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.2/python3.7/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 7500: auc = 0.7682979, eval_accuracy = 0.7682, f1_score = 0.7667773, false_negatives = 2459.0, false_positives = 2177.0, global_step = 7500, loss = 0.8033782, precision = 0.7778118, recall = 0.7560516, true_negatives = 7743.0, true_positives = 7621.0


INFO:tensorflow:Saving dict for global step 7500: auc = 0.7682979, eval_accuracy = 0.7682, f1_score = 0.7667773, false_negatives = 2459.0, false_positives = 2177.0, global_step = 7500, loss = 0.8033782, precision = 0.7778118, recall = 0.7560516, true_negatives = 7743.0, true_positives = 7621.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7500: bert/output/model.ckpt-7500


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7500: bert/output/model.ckpt-7500


{'auc': 0.7682979,
 'eval_accuracy': 0.7682,
 'f1_score': 0.7667773,
 'false_negatives': 2459.0,
 'false_positives': 2177.0,
 'global_step': 7500,
 'loss': 0.8033782,
 'precision': 0.7778118,
 'recall': 0.7560516,
 'true_negatives': 7743.0,
 'true_positives': 7621.0}

# Now, if you want to do some Predictions

In [ ]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
pred_sentences = [
    "Like and subscribe!",
    "That was great!",
    "That was Bad!",
    "Well Done!"
]

In [ ]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:Writing example 0 of 4


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from bert/output/model.ckpt-7500


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [ ]:
predictions

[('Like and subscribe!',
  array([-4.196163  , -0.01516762], dtype=float32),
  'Positive'),
 ('That was great!',
  array([-6.2471666e+00, -1.9377756e-03], dtype=float32),
  'Positive'),
 ('That was Bad!', array([-0.3937208, -1.1225228], dtype=float32), 'Negative'),
 ('Well Done!',
  array([-5.5058508e+00, -4.0711625e-03], dtype=float32),
  'Positive')]

In [ ]:
"""Now, if you are curious on how to conduct the pretraining phase i.e (MLM and NSP)
you will need to run the following: I literally copied and pasted this part from
the google repo; of course you will need to change a few parameters..."""

# Create Pretraining Data

# python create_pretraining_data.py \
#   --input_file=./sample_text.txt \
#   --output_file=/tmp/tf_examples.tfrecord \
#   --vocab_file=$BERT_BASE_DIR/vocab.txt \
#   --do_lower_case=True \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=5

# Create Context and Language understanding

# !python bert/run_pretraining.py \
#   --input_file=/tmp/tf_examples.tfrecord \
#   --output_dir=/tmp/pretraining_output \
#   --do_train=True \
#   --do_eval=True \
#   --bert_config_file=$BERT_BASE_DIR/bert_config.json \
#   --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
#   --train_batch_size=32 \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --num_train_steps=20 \
#   --num_warmup_steps=10 \
#   --learning_rate=2e-5